# Function:Calculate type distribution [4.nextclade/china_nextclade_good.tsv]

## 1.World-clade

### extract clade

In [1]:
# extract clade
! awk -F'\t' '{print $2 "\t" $3 "\t" $4}' 4.nextclade/world_nextclade_good.tsv > 6.clade/world_clade.tsv

### extract date

In [2]:
# split
n = 0
with open('6.clade/world_clade.tsv', 'r') as file, open('6.clade/world_clade_updated.tsv', 'w') as f_out:
    for line in file:
        columns = line.strip().split('\t')

        if len(columns) < 3:
            n += 1
            continue
        collection_date = columns[0].split('|')[1]
        f_out.write(columns[0] + '\t' + collection_date + '\t' + columns[1] + '\t' + columns[2] + '\n')
    print(n)


0


### mark BA.5.2.48/BF.7

In [1]:
import pandas as pd
df = pd.read_csv('6.clade/world_clade_updated.tsv', sep='\t', header=None, index_col=None)

for index, row in df.iterrows():
    if row[2] == '22B':
        if row[3].startswith(('DY.1', 'DY.2', 'DY.3', 'DY.4', 'BA.5.2.48', 'DY.1.1')):
            df.at[index, 2] = '22B(BA.5.2.48)'
        elif row[3].startswith('BF.7'):
            df.at[index, 2] = '22B(BF.7)'
        else:
            df.at[index, 2] = '22B(Other)'
df.to_csv("6.clade/world_clade_marked.tsv", sep="\t", index=None, header=None)


In [3]:
! cut -f 3 6.clade/world_clade_marked.tsv | sort | uniq -c

     25 19A
      4 19B
      5 20A
      9 20B
      3 20C
      1 20J
      6 21J
     83 21K
   5948 21L
  12637 22A
    689 22B(BA.5.2.48)
  53925 22B(BF.7)
 228276 22B(Other)
     16 22C
  49283 22D
 327337 22E
  42612 22F
 209597 23A
  27190 23B
  35789 23C
  53252 23D
  10260 23E
   5970 23F
    933 23G
     15 23H
     22 23I
  17548 recombinant


### calculate count&frequeny

In [4]:
import pandas as pd
df = pd.read_csv('6.clade/world_clade_marked.tsv', sep='\t', header=None, index_col=None)

############# count ###############
# Convert to datetime
df[1] = pd.to_datetime(df[1], format='%Y-%m-%d', errors='coerce')

# Group by week and clade
weekly_clade_counts = df.groupby([pd.Grouper(key=1, freq='W'), 2]).size().unstack(fill_value=0)
weekly_clade_counts.to_csv('6.clade/world_weekly_clade_counts.csv')

############ frequency ################
weekly_clade_distribution = weekly_clade_counts.div(weekly_clade_counts.sum(axis=1), axis=0)
weekly_clade_distribution.to_csv('6.clade/world_weekly_clade_distribution.csv')

## 2.China_clade

### extract clade&date

In [7]:
# clade data
! awk -F'\t' '{print $2 "\t" $3 "\t" $4}' 4.nextclade/china_nextclade_good.tsv > 6.clade/china_clade.tsv

### Delete Cantonese sequences

In [6]:

# split
m = 0
n = 0
with open('6.clade/china_clade.tsv', 'r') as file, open('6.clade/guangdong_clade_updated.tsv', 'w') as gd_out, open('6.clade/china_clade_updated.tsv', 'w') as f_out:
    for line in file:
        columns = line.strip().split('\t')
        collection_date = columns[0].split('|')[1]

        # Delete Cantonese sequences
        if columns[0].startswith('hCoV-19/Guangdong'):
            m += 1
            gd_out.write(columns[0] + '\t' + collection_date + '\t' + columns[1] + '\t' + columns[2] + '\n')
            continue

        if len(columns) < 3:
            n += 1
            continue
        
        f_out.write(columns[0] + '\t' + collection_date + '\t' + columns[1] + '\t' + columns[2] + '\n')
    print(m)
    print(n)

4800
0


### mark BA.5.2.48/BF.7

In [9]:
### mark BA.5.2.48/BF.7
import pandas as pd
df = pd.read_csv('6.clade/china_clade_updated.tsv', sep='\t', header=None, index_col=None)

for index, row in df.iterrows():
    if row[2] == '22B':
        if row[3].startswith(('DY.1', 'DY.2', 'DY.3', 'DY.4', 'BA.5.2.48', 'DY.1.1')):
            df.at[index, 2] = '22B(BA.5.2.48)'
        elif row[3].startswith('BF.7'):
            df.at[index, 2] = '22B(BF.7)'
        else:
            df.at[index, 2] = '22B(Other)'
df.to_csv("6.clade/china_clade_marked.tsv", sep="\t", index=None, header=None)


In [10]:
! cut -f 3 6.clade/china_clade_marked.tsv | sort | uniq -c

      9 21L
      2 22A
   9945 22B(BA.5.2.48)
   5203 22B(BF.7)
   1065 22B(Other)
      5 22C
    665 22D
    188 22E
   2616 22F
   1350 23A
   3485 23B
     20 23C
   4170 23D
    285 23E
   3032 23F
      4 23G
     57 23H
     41 recombinant


### calculate count&frequency

In [11]:

import pandas as pd
df = pd.read_csv('6.clade/china_clade_marked.tsv', sep='\t', header=None, index_col=None)

############# number ###############
# Convert to datetime
df[1] = pd.to_datetime(df[1], format='%Y-%m-%d', errors='coerce')

# Group by week and clade
weekly_clade_counts = df.groupby([pd.Grouper(key=1, freq='W'), 2]).size().unstack(fill_value=0)
weekly_clade_counts.to_csv('6.clade/china_weekly_clade_counts.csv')

############ frequency ################
weekly_clade_distribution = weekly_clade_counts.div(weekly_clade_counts.sum(axis=1), axis=0)
weekly_clade_distribution.to_csv('6.clade/china_weekly_clade_distribution.csv')

## 3.Guangdong-clade

Sequence uploaded to gisaid by Guangdong Province

Processing sequences in Guangdong: 9.gd_seq/gd_seq.ipynb